In [1]:
import os, sys
import time
import numpy as np
import pandas as pd
import random
from scipy import stats as st
import itertools
import operator

import torch

from tqdm.notebook import trange
from tqdm import tqdm

# Init steps

In [2]:
# get currently working directory
base_dir = os.getcwd()

# load functions from other notebooks
helpers_file = os.path.join(base_dir, 'helpers.ipynb')
%run $helpers_file

# # load the autoreload extension
# %load_ext autoreload
# # Set extension to reload modules every time before executing code
# %autoreload 2

In [3]:
for p in ['../spotlight_ext']:
    module_path = os.path.abspath(os.path.join(base_dir, p))
    if module_path not in sys.path:
        sys.path.append(module_path)

random_state = np.random.RandomState(2020)

In [4]:
## !jupyter nbconvert budget_strategies.ipynb --no-input --no-prompt --to pdf
# os.system("jupyter nbconvert budget_strategies.ipynb --no-input --no-prompt --to pdf")
# os.system("jupyter nbconvert budget_strategies.ipynb --config ~/.jupyter/jupyter_nbconvert_config.py --to slides")

# Prepare models/datasets

In [5]:
# implicit_model = load_model('implicit_factorization')
lstm_model = load_model(model_type='entire')
pooling_model = load_model('pooling')

pretrained_models = {
    'lstm': lstm_model,
    'pooling': pooling_model,
}

In [6]:
from spotlight.cross_validation import random_train_test_split
from spotlight.datasets.movielens import get_movielens_dataset

# get dataset
dataset = get_movielens_dataset(variant='1M')
train, test = random_train_test_split(dataset, random_state=random_state)

max_sequence_length = 20
train = train.to_sequence(max_sequence_length=max_sequence_length)
test = test.to_sequence(max_sequence_length=max_sequence_length)

In [7]:
pooling_sims_matrix = gpu_embeddings_to_cosine_similarity_matrix(
    pooling_model._net.item_embeddings(
        torch.arange(0, dataset.num_items, dtype=torch.int64)
    )).detach().numpy()

jaccard_sims_matrix = compute_sim_matrix(dataset, 'jaccard')

  0%|          | 0/6040 [00:00<?, ?it/s]

# Various implemented Strategies

In [8]:
class BaseStrategy:
    class_name = None

    def __init__(self, item, interactions, max_length, init_budget, model=None, random_pick=False):

        self.target_item = item
        self.original_interactions = interactions
        self.max_length = max_length
        self.visited_ = set()
        self.model = model
        self.last_comb_cost = 0
        self.random_pick = random_pick
        self.top_k = 10

        self.budget = init_budget

    def next_comb(self, reverse=False):
        raise NotImplementedError

    def _get_pos(self, number):
        bits = []
        for i, c in enumerate(bin(number)[:1:-1], 1):
            if c == '0':
                bits.append(i)
        return bits

    def reset_costs(self):
        self.last_comb_cost = 0

    def get_init_budget(self):
        return self.budget

In [9]:
class RandomSelection(BaseStrategy):
    class_name = 'Random'

    def __init__(self, item, interactions, max_sequence_length, init_budget, model):
        super().__init__(item, interactions, max_sequence_length, init_budget)

    def _next_item(self):
        self.budget -= 1

        #         number = np.random.choice(np.setdiff1d(range(1, pow(2, self.max_length)), self.visited_))
        number = random.sample(range(1, pow(2, self.max_length)), 1)[0]
        while number in self.visited_:
            number = random.sample(range(1, pow(2, self.max_length)), 1)[0]
        self.visited_.add(number)
        return number

    def next_comb(self, reverse=False):
        number = self._next_item()

        bits = self._get_pos(number)
        seq = np.delete(self.original_interactions, bits)

        return (seq, self.budget)

In [10]:
class MostSimilarSelection(BaseStrategy):
    class_name = 'Sim-Matrix'

    supported_sim_matrix = {
        'pooling': pooling_sims_matrix,
        'jaccard': jaccard_sims_matrix
    }

    def __init__(self, item, interactions, max_sequence_length, model, sim_type='pooling'):
        super().__init__(item, interactions, max_sequence_length)

        self.visited_.add(0)
        self.reverse_checks = []
        self.is_materialized = False

        self._get_sim_ranking(sim_type)

    def next_comb(self, reverse=False):
        if reverse:
            self._materialize_list()
            selected_item_indices = self.reverse_checks.pop(
                random.randrange(len(self.reverse_checks)) if self.random_pick else 0
            ) if len(self.reverse_checks) else []
        else:
            self.visited_.add(max(self.visited_) + 1)
            selected_item_indices = np.where(np.isin(
                self.rk_items,
                list(set(self.rk_items).difference(set(self.visited_)))
            ))[0]
        seq = self.original_interactions[selected_item_indices] if len(selected_item_indices) else None
        return seq

    def _get_sim_ranking(self, sim_type):
        ranked_items = st.rankdata(self.supported_sim_matrix[sim_type][self.target_item, self.original_interactions])
        self.rk_items = self.max_length - ranked_items + 1

    def _materialize_list(self):
        if not self.is_materialized:
            psize = len(self.visited_) - 1  # do not consider initial added zero value
            # do not take account none/all excluded interacted items
            prods = sorted(list(map(list, itertools.product([0, 1], repeat=psize)))[1:-1], key=sum)
#             last_item_indices = np.where(np.isin(
#                 self.rk_items,
#                 list(set(self.rk_items).difference(set(self.visited_)))
#             ))

            lvisited_ = np.asarray(list(self.visited_))[1:]
            for p in prods:
                self.reverse_checks.append(np.where(np.isin(
                    self.rk_items,
                    list(set(self.rk_items).difference(lvisited_[np.nonzero(np.multiply(p, lvisited_))])))
                ))

            self.is_materialized = True

In [11]:
class MostSimilarSelectionByJaccard(MostSimilarSelection):
    class_name = 'Jaccard-on-Sim-Matrix'

    def __init__(self, item, interactions, max_sequence_length, model):
        super().__init__(item, interactions, max_sequence_length, model, 'jaccard')

In [12]:
class RandomMostSimilarSelection(MostSimilarSelection):
    def __init__(self, item, interactions, max_sequence_length, model):
        super().__init__(item, interactions, max_sequence_length, model)

        self.random_pick = True

In [13]:
class LossSimilarSelection(BaseStrategy):
    class_name = 'BFS'

#     def next_comb(self, reverse=False):
#         if reverse:
#             self._materialize_list()
#         else:
#             self._get_sim_ranking()

#         seq = np.ma.compressed(self.ma_arr)
#         return seq if len(seq) else None

#     def compute_losses(self, inv_mask=False):
#         res = []
#         self.last_comb_cost = 0

#         m_mask = np.ma.getmask(self.ma_arr).copy()
#         tmp_ma_arr = np.ma.masked_array(self.original_interactions, mask=np.logical_not(m_mask) if inv_mask else m_mask)
#         for idx in tmp_ma_arr.nonzero()[0]:
#             m_mask[idx] = not m_mask[idx]
#             if np.any(np.invert(m_mask)):
#                 perm = np.ma.compressed(np.ma.masked_array(self.original_interactions, mask=m_mask))

#                 # predict next top-k items about to be selected
#                 preds = self.model.predict(perm)
#                 preds[perm] = -StaticVars.FLOAT_MAX
#                 t_score = preds[self.target_item]
#                 res.append([t_score, idx])

#                 self.last_comb_cost += 1
# #             else: res.append([0, -1])

#             m_mask[idx] = not m_mask[idx]

#         return res

#     def _materialize_list(self):
#         if not self.is_materialized:
#             m_mask = np.ma.getmask(self.ma_arr).copy()
#             psize = sum(m_mask)
#             # do not take account none/all excluded interacted items
#             prods = sorted(list(map(list, itertools.product([0, 1], repeat=psize)))[1:-1], key=sum)

#             lvisited_ = np.where(m_mask == True)[0]
#             for p in prods:
#                 curr_mask = np.ma.getmaskarray(self.ma_arr).copy()
#                 curr_mask[lvisited_[np.nonzero(np.multiply(p, lvisited_))]] = 0

#                 self.reverse_checks.append(curr_mask)

#             self.is_materialized = True

#         m_mask = self.reverse_checks.pop(
#             random.randrange(len(self.reverse_checks)) if self.random_pick else 0
#         ) if len(self.reverse_checks) else True
#         self.ma_arr = np.ma.masked_array(self.original_interactions, mask=m_mask)

    def __init__(self, item, interactions, max_sequence_length, init_budget, model, early_term=False):
        super().__init__(item, interactions, max_sequence_length, init_budget, model)

        self.q = Queue()
        self.q.enqueue(([False] * len(self.original_interactions), StaticVars.INT_MAX, 0))

        self.thres = len(self.original_interactions) + 1
        self.early_termination = early_term

    def _update_queue(self, is_solved):
        self.compute_loss(is_solved)

    def _next_item(self):
        mask, t_score, is_solved = self.q.dequeue()
        while self.early_termination and sum(mask) == self.thres:
            q_data = self.q.dequeue()
            if q_data is None: break

            mask, t_score, is_solved = q_data

        if is_solved == 2:
            t_score, kth_score = self.get_score(mask)

            if (t_score / kth_score) < 1: self.thres = sum(mask)

        return (is_solved, mask, self.budget)

    def next_comb(self, reverse=False):
        budget = self.budget

        if self.q.size() > 0:
            solved_flag, item_mask, budget = self._next_item()
            self.ma_arr = np.ma.masked_array(self.original_interactions, mask=item_mask.copy())
            self._update_queue(solved_flag)
        else: self.ma_arr = np.ma.masked_array(self.original_interactions, mask=True)

        seq = np.ma.compressed(self.ma_arr)
        return (seq, budget) if len(seq) else (None, budget)

    def compute_loss(self, is_solved=False):
        self.last_comb_cost = 0

        if not is_solved: self.search(forward=True, s=is_solved)
        else: self.search(forward=False, s=is_solved)

    def search(self, forward=True, s=False):
        m_mask = np.ma.getmask(self.ma_arr).copy()
        valid_items = np.where(np.logical_not(m_mask) if forward else m_mask)[0]
        if valid_items.size > 1:
            for idx in valid_items:
                m_mask[idx] = not m_mask[idx]
                self.add(m_mask, s)
                m_mask[idx] = not m_mask[idx]

    def get_score(self, d):
        perm = np.ma.compressed(np.ma.masked_array(self.original_interactions, mask=d))

        self.budget -= 1
        # predict next top-k items about to be selected
        preds = self.model.predict(perm)
        preds[perm] = -StaticVars.FLOAT_MAX
        rk_data = st.rankdata(-preds, method='ordinal')

        return (preds[self.target_item], preds[(rk_data == self.top_k).nonzero()][0])

    def add(self, d, s):
        mask_to_int = int(''.join(map(str, d.astype(int))), 2)
        if (mask_to_int not in self.visited_) and (self.budget > 0):
            perm = np.ma.compressed(np.ma.masked_array(self.original_interactions, mask=d))

            if not s:
                t_score, kth_score = self.get_score(d)

                if self.q.size() == 0: self.q.enqueue((d.copy(), t_score, 1 if (t_score / kth_score) < 1 else 0))

                if t_score < self.q.get(0)[1]:  # get only the assigned score
                    self.q.setter(0, (d.copy(), t_score, 1 if (t_score / kth_score) < 1 else 0))
            else:
                self.q.enqueue((d.copy(), StaticVars.INT_MAX, 2))

            self.visited_.add(mask_to_int)

In [14]:
class DFSwithLossSelection(LossSimilarSelection):
    class_name = 'DFS'

    def __init__(self, item, interactions, max_sequence_length, model):
        super().__init__(item, interactions, max_sequence_length, model)

        self.candidate_solutions = Stack()

    def _get_sim_ranking(self):
        res = self.compute_losses()

        m_mask = np.ma.getmask(self.ma_arr).copy()
        m_mask[min(res, key=lambda item: item[0])[1]] = True
        self.ma_arr = np.ma.masked_array(self.original_interactions, mask=m_mask)

    def _materialize_list(self):
        m_mask = np.ma.getmask(self.ma_arr).copy()

        if np.sum(m_mask) > 1:
            res = self.compute_losses(inv_mask=True)

            for idx in sorted(res, key=lambda item: item[0], reverse=True):
                m_mask[idx[1]] = False
                self.candidate_solutions.push(m_mask.copy())
                m_mask[idx[1]] = True

        m_mask = self.candidate_solutions.pop()
        while m_mask is not None and int(''.join(map(str, m_mask.astype(int))), 2) in self.visited_:
            m_mask = self.candidate_solutions.pop()

        self.visited_.add(0 if m_mask is None else int(''.join(map(str, m_mask.astype(int))), 2))
        self.ma_arr = np.ma.masked_array(self.original_interactions, mask=True if m_mask is None else m_mask)

In [15]:
class RandomLossSimilarSelection(LossSimilarSelection):
    def __init__(self, item, interactions, max_sequence_length, model):
        super().__init__(item, interactions, max_sequence_length, model)

        self.random_pick = True

In [16]:
class FixedRankingLossSimilarSelection(LossSimilarSelection):
    def __init__(self, item, interactions, max_sequence_length, model):
        super().__init__(item, interactions, max_sequence_length, model)

        self.rk_items = []
        self._get_sim_ranking()

    def next_comb(self, reverse=False):
        if reverse:
            self._materialize_list()
        else:
            m_mask = np.ma.getmask(self.ma_arr).copy()
            m_mask[self.rk_items.pop(0)] = True
            self.ma_arr = np.ma.masked_array(self.original_interactions, mask=m_mask)

        seq = np.ma.compressed(self.ma_arr)
        return seq if len(seq) else None

    def _get_sim_ranking(self):
        res = self.compute_losses()

        ranked_items = np.asarray(res).argsort(axis=0)
        self.rk_items = [item[0] for item in ranked_items]

In [17]:
import heapq as hq


class BestFSLossSelection(LossSimilarSelection):
    def __init__(self, item, interactions, max_sequence_length, model):
        super().__init__(item, interactions, max_sequence_length, model)

        self.set = set()
        self.tiebraker = itertools.count()

    def _materialize_list(self):
        m_mask = np.ma.getmask(self.ma_arr).copy()

        if not self.is_materialized:
            if np.sum(m_mask) > 1:
                res = self.compute_losses(inv_mask=True)

                for idx in res:
                    m_mask[idx[1]] = False
                    self.reverse_checks.append((idx[0], next(self.tiebraker), m_mask.copy()))
                    self.set.add(int(''.join(map(str, m_mask.copy().astype(int))), 2))
                    m_mask[idx[1]] = True

                hq.heapify(self.reverse_checks)

            self.is_materialized = True

        _, _, m_mask = hq.heappop(self.reverse_checks) if len(self.reverse_checks) > 0 else (None, None, True)
        self.ma_arr = np.ma.masked_array(self.original_interactions, mask=m_mask)

        if np.sum(m_mask) > 1:
            m_mask = np.ma.getmask(self.ma_arr).copy()
            res = self.compute_losses(inv_mask=True)

            for idx in res:
                m_mask[idx[1]] = False
                self.add(m_mask.copy(), idx[0])
                m_mask[idx[1]] = True

    def add(self, d, pri):
        mask_to_int = int(''.join(map(str, d.astype(int))), 2)
        if mask_to_int not in self.set:
            hq.heappush(self.reverse_checks, (pri, next(self.tiebraker), d))
            self.set.add(mask_to_int)

In [18]:
import heapq as hq


class TopDownBestFSLossSelection(LossSimilarSelection):
    def __init__(self, item, interactions, max_sequence_length, model):
        super().__init__(item, interactions, max_sequence_length, model)

        self.best_score_per_cardinality = [StaticVars.FLOAT_MAX] * self.max_length
        self.set = set()
        self.tiebraker = itertools.count()

    def set_score(self, cardinality, target_score, kth_score):
        reverse_search = False
        score = target_score / kth_score
        if score < self.best_score_per_cardinality[cardinality]:
            self.best_score_per_cardinality[cardinality] = score

            if score > 1.0: reverse_search = True

        return reverse_search

    def _materialize_list(self):
        m_mask = np.ma.getmask(self.ma_arr).copy()

        if not self.is_materialized:
            if np.sum(m_mask) > 1:
                res = self.compute_losses(inv_mask=True)

                for idx in res:
                    m_mask[idx[1]] = False
                    self.reverse_checks.append((idx[0], next(self.tiebraker), m_mask.copy()))
                    self.set.add(int(''.join(map(str, m_mask.copy().astype(int))), 2))
                    m_mask[idx[1]] = True

                hq.heapify(self.reverse_checks)

            self.is_materialized = True

        _, _, m_mask = hq.heappop(self.reverse_checks) if len(self.reverse_checks) > 0 else (None, None, True)
        self.ma_arr = np.ma.masked_array(self.original_interactions, mask=m_mask)

        if np.sum(m_mask) > 1:
            m_mask = np.ma.getmask(self.ma_arr).copy()
            res = self.compute_losses(inv_mask=True)

            for idx in res:
                m_mask[idx[1]] = False
                self.add(m_mask.copy(), idx[0])
                m_mask[idx[1]] = True

    def add(self, d, pri):
        mask_to_int = int(''.join(map(str, d.astype(int))), 2)
        if mask_to_int not in self.set:
            hq.heappush(self.reverse_checks, (pri, next(self.tiebraker), d))
            self.set.add(mask_to_int)

In [19]:
class DFSwithFixedRankingLossSelection(FixedRankingLossSimilarSelection):

    def __init__(self, item, interactions, max_sequence_length, model):
        super().__init__(item, interactions, max_sequence_length, model)

    def next_comb(self, reverse=False):
        if reverse:
            self._materialize_list()
        else:
            m_mask = np.ma.getmask(self.ma_arr).copy()
            m_mask[self.rk_items.pop(0)] = True
            self.ma_arr = np.ma.masked_array(self.original_interactions, mask=m_mask)

        seq = np.ma.compressed(self.ma_arr)
        return seq if len(seq) else None

    def _materialize_list(self):
        m_mask = np.ma.getmask(self.ma_arr).copy()

        if np.sum(m_mask) > 1:
            res = self.compute_losses(inv_mask=True)

            if not self.is_materialized:
                for idx in sorted(res, key=lambda item: item[0]):
                    m_mask[idx[1]] = False
                    self.reverse_checks.append(m_mask.copy())
                    m_mask[idx[1]] = True

                self.is_materialized = True
            else:
                m_mask[min(res, key=lambda item: item[0])[1]] = False
                self.reverse_checks.insert(0, m_mask)

        m_mask = self.reverse_checks.pop(
            random.randrange(len(self.reverse_checks)) if self.random_pick else 0
        ) if len(self.reverse_checks) else True
        self.ma_arr = np.ma.masked_array(self.original_interactions, mask=m_mask)

In [20]:
import heapq as hq


class BestFSFixedLossSelection(FixedRankingLossSimilarSelection):
    def __init__(self, item, interactions, max_sequence_length, model):
        super().__init__(item, interactions, max_sequence_length, model)

        self.best_score_per_cardinality = [-StaticVars.FLOAT_MAX] * self.max_length
        self.set = set()
        self.tiebraker = itertools.count()

#     is not currently used
#     def set_score(self, cardinality, target_score, kth_score):
#         is_updated = False
#         if self.best_score_per_cardinality[cardinality] > (target_score / kth_score):
#             self.best_score_per_cardinality[cardinality] = target_score / kth_score
#             is_updated = True

#         return is_updated

    def _materialize_list(self):
        m_mask = np.ma.getmask(self.ma_arr).copy()

        if not self.is_materialized:
            if np.sum(m_mask) > 1:
                res = self.compute_losses(inv_mask=True)

                for idx in res:
                    m_mask[idx[1]] = False
                    self.reverse_checks.append((idx[0], next(self.tiebraker), m_mask.copy()))
                    self.set.add(int(''.join(map(str, m_mask.copy().astype(int))), 2))
                    m_mask[idx[1]] = True

                hq.heapify(self.reverse_checks)

            self.is_materialized = True

        _, _, m_mask = hq.heappop(self.reverse_checks) if len(self.reverse_checks) > 0 else (None, None, True)
        self.ma_arr = np.ma.masked_array(self.original_interactions, mask=m_mask)

        if np.sum(m_mask) > 1:
            m_mask = np.ma.getmask(self.ma_arr).copy()
            res = self.compute_losses(inv_mask=True)

            for idx in res:
                m_mask[idx[1]] = False
                self.add(m_mask.copy(), idx[0])
                m_mask[idx[1]] = True

    def add(self, d, pri):
        mask_to_int = int(''.join(map(str, d.astype(int))), 2)
        if mask_to_int not in self.set:
            hq.heappush(self.reverse_checks, (pri, next(self.tiebraker), d))
            self.set.add(mask_to_int)

In [60]:
class BiDirectionalSelection(BaseStrategy):
    class_name = 'BiDirectional'

    def __init__(self, item, interactions, max_sequence_length, init_budget, model, weights=(1, 0), alpha=0.9, normalization='default'):
        super().__init__(item, interactions, max_sequence_length, init_budget, model)

        self.tiebraker = itertools.count()
        self.q = [(1, StaticVars.INT_MAX, next(self.tiebraker), [False] * len(self.original_interactions), self.budget)]
        hq.heapify(self.q)

#         self.w_loss, self.w_custom = weights if len(weights) == 2 else (1, 0)
        self.alpha = alpha
        self.norm = normalization

    def _update_queue(self, is_solved):
        self.compute_loss(is_solved)

    def _next_item(self):
        is_solved, _, _, mask, budget = hq.heappop(self.q)
        return (is_solved, mask, budget)

    def next_comb(self, reverse=False):
        budget = self.budget
        if self.q:
            solved_flag, item_mask, budget = self._next_item()
            self.ma_arr = np.ma.masked_array(self.original_interactions, mask=item_mask.copy())
            self._update_queue(solved_flag)
        else: self.ma_arr = np.ma.masked_array(self.original_interactions, mask=True)

        seq = np.ma.compressed(self.ma_arr)
        return (seq, budget) if len(seq) else (None, budget)

    def compute_loss(self, is_solved=False):
        self.search(forward=True, s=is_solved)
        self.search(forward=False, s=is_solved)

    def search(self, forward=True, s=False):
        m_mask = np.ma.getmask(self.ma_arr).copy()
        valid_items = np.where(np.logical_not(m_mask) if forward else m_mask)[0]
        if valid_items.size > 1:
            for idx in valid_items:
                m_mask[idx] = not m_mask[idx]
                self.add(m_mask, s)
                m_mask[idx] = not m_mask[idx]

    def get_custom_score(self, c):
#         return self.w_custom * (c / self.max_length)
        return c / self.max_length

    def get_score(self, d):
        self.budget -= 1

        # predict next top-k items about to be selected
        perm = np.ma.compressed(np.ma.masked_array(self.original_interactions, mask=d))
        preds = self.model.predict(perm)

        if self.norm == 'kth_norm':
            preds[perm] = -StaticVars.FLOAT_MAX
            rk_data = st.rankdata(-preds, method='ordinal')

            t_score = preds[self.target_item] / preds[(rk_data == self.top_k).nonzero()][0]
        elif self.norm == 'rescale':
            preds[perm] = -StaticVars.FLOAT_MAX
            rk_data = st.rankdata(-preds, method='ordinal')

            max_val = rk_data[0]
            min_val = rk_data[-1]
            t_score = (max_val - preds[self.target_item]) / (max_val - min_val)
        else:  # default case
            tensor = F.softmax(torch.from_numpy(preds).float(), dim=0)
            preds = tensor.numpy()
            preds[perm] = -StaticVars.FLOAT_MAX

            t_score = preds[self.target_item]

        return self.alpha * t_score + (1 - self.alpha) * self.get_custom_score(np.sum(d))

    def add(self, d, s):
        mask_to_int = int(''.join(map(str, d.astype(int))), 2)
        if (mask_to_int not in self.visited_) and (self.budget > 0):
            t_score = self.get_score(d)
            hq.heappush(self.q, (int(not s), t_score, next(self.tiebraker), d.copy(), self.budget))

            self.visited_.add(mask_to_int)

In [61]:
class BruteForceSelection(BaseStrategy):
    class_name = 'BruteForce'

    def __init__(self, item, interactions, max_sequence_length, init_budget, model):
        super().__init__(item, interactions, max_sequence_length, init_budget, model)

        self.q = Queue()
        self.q.enqueue(([False] * len(self.original_interactions), self.budget))

    def _expand_queue(self):
        m_mask = np.ma.getmask(self.ma_arr).copy()
        valid_items = np.where(np.logical_not(m_mask))[0]
        if valid_items.size > 1:
            for idx in valid_items:
                m_mask[idx] = not m_mask[idx]
                self.add(m_mask)
                m_mask[idx] = not m_mask[idx]

    def _next_item(self):
        mask, budget = self.q.dequeue()
        return (mask, budget)

    def next_comb(self, reverse=False):
        budget = self.budget

        if reverse: self.q.clear()

        if self.q.size() > 0:
            item_mask, budget = self._next_item()
            self.ma_arr = np.ma.masked_array(self.original_interactions, mask=item_mask.copy())
            self._expand_queue()
        else:
            self.ma_arr = np.ma.masked_array(self.original_interactions, mask=True)

        seq = np.ma.compressed(self.ma_arr)
        return (seq, budget) if len(seq) else (None, budget)

    def add(self, d):
        mask_to_int = int(''.join(map(str, d.astype(int))), 2)
        if (mask_to_int not in self.visited_) and (self.budget > 0):
            self.budget -= 1
            self.q.enqueue((d.copy(), self.budget))
            self.visited_.add(mask_to_int)

In [68]:
class ComboSelection(BiDirectionalSelection):
    class_name = 'Combo'

    def __init__(self, item, interactions, max_sequence_length, init_budget, model, weights=(1, 0), alpha=0.9, normalization='default'):
        super().__init__(item, interactions, max_sequence_length, init_budget, model, weights, alpha, normalization)

        self.alpha = 1

        self.q_init = Queue()
        self.q_init.enqueue((StaticVars.INT_MAX, [False] * len(self.original_interactions), self.budget))
        self.init_queue()

        self.tiebraker = itertools.count()
        self.q = []
        hq.heapify(self.q)

        self.alpha = alpha

    def init_queue(self):
        _, m_mask, budget = self.q_init.dequeue()
        m_mask = np.asarray(m_mask)

        valid_items = np.where(np.logical_not(m_mask))[0]
        for idx in valid_items:
            m_mask[idx] = not m_mask[idx]

            mask_to_int = int(''.join(map(str, m_mask.astype(int))), 2)
            if (mask_to_int not in self.visited_) and (self.budget > 0):
                t_score = self.get_score(m_mask)
                self.q_init.enqueue((t_score, m_mask.copy(), self.budget))

                self.visited_.add(mask_to_int)

            m_mask[idx] = not m_mask[idx]

        pair_combs = []
        for c in itertools.combinations(range(len(self.original_interactions)), 2):
            m = [False] * len(self.original_interactions)
            m[c[0]], m[c[1]] = not m[c[0]], not m[c[1]]
            pair_combs.append((self.q_init.get(c[0])[0] + self.q_init.get(c[1])[0], m.copy()))

        pair_combs.sort(key=operator.itemgetter(0))
        for c in pair_combs:
            self.budget -= 1
            self.q_init.enqueue((0, c[1], self.budget))

    def next_comb(self, reverse=False):
        budget = self.budget

        if self.q_init.size() > 0:
            s, item_mask, budget = self.q_init.dequeue()
            item_mask = np.asarray(item_mask)
            solved_flag = False
            self.ma_arr = np.ma.masked_array(self.original_interactions, mask=item_mask.copy())

            self.add(item_mask, False)
        elif self.q:
            solved_flag, item_mask, budget = self._next_item()
            self.ma_arr = np.ma.masked_array(self.original_interactions, mask=item_mask.copy())

            self._update_queue(solved_flag)
        else: self.ma_arr = np.ma.masked_array(self.original_interactions, mask=True)

        seq = np.ma.compressed(self.ma_arr)
        return (seq, budget) if len(seq) else (None, budget)

In [69]:
def get_backend_strategy(backend):
    if 'random' == backend:
        return RandomSelection
    elif 'most_sim' == backend:
        return MostSimilarSelection
    elif 'most_sim_jaccard' == backend:
        return MostSimilarSelectionByJaccard
    elif 'bfs' == backend:
        return LossSimilarSelection
    elif 'random_most_sim' == backend:
        return RandomMostSimilarSelection
    elif 'random_loss_sim' == backend:
        return RandomLossSimilarSelection
    elif 'fixed_loss_sim' == backend:
        return FixedRankingLossSimilarSelection
    elif 'dfs_loss_sim' == backend:
        return DFSwithLossSelection
    elif 'dfs_fixed_loss_sim' == backend:
        return DFSwithFixedRankingLossSelection
    elif 'bestFS_loss' == backend:
        return BestFSLossSelection
    elif 'bestFS_fixed_loss' == backend:
        return BestFSFixedLossSelection
    elif 'topdown_loss' == backend:
        return TopDownBestFSLossSelection
    elif 'bidirectional' == backend:
        return BiDirectionalSelection
    elif 'brute_force' == backend:
        return BruteForceSelection
    elif 'combo' == backend:
        return ComboSelection
    else: print('Unknown strategy')

# Execution of implemented strategies

## Strategies inputs

In [27]:
model_applied = 'lstm'

## **Random**

In [71]:
backend = 'random'
random_cfs = [
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [1, 3, 5, 7], no_users=6000, init_budget=1000),
#     _find_cfs(test, pretrained_models['pooling'], [3, 5, 7], no_users=500, backend='random', init_budget=1000)
]

%store random_cfs

target position loop:   0%|          | 0/4 [00:00<?, ?it/s]

The backend used is: Random


target position loop: 20it [54:45, 164.26s/it]             

target position loop: 30it [1:49:17, 213.14s/it]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## **Most Similar**

In [359]:
cosine_on_embeddings_cfs = [
    _find_cfs(test, pretrained_models['lstm'], [3, 5, 7], no_users=500, backend='most_sim', init_budget=1000),
    _find_cfs(test, pretrained_models['pooling'], [3, 5, 7], no_users=500, backend='most_sim', init_budget=1000)
]
jaccard_on_embeddings_cfs = [
    _find_cfs(test, pretrained_models['lstm'], [3, 5, 7], no_users=500, backend='most_sim_jaccard', init_budget=1000),
    _find_cfs(test, pretrained_models['pooling'], [3, 5, 7], no_users=500, backend='most_sim_jaccard', init_budget=1000),
]

%store cosine_on_embeddings_cfs
%store jaccard_on_embeddings_cfs

## Utilize ylosses for similarities

In [104]:
backend = 'bfs'
bfs_yloss_cfs = [
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [1, 3, 5, 7], no_users=6000, init_budget=1000),
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [1, 3, 5, 7], no_users=6000, init_budget=1000, early_term=True),
#     _find_cfs(test, pretrained_models['pooling'], [3, 5, 7], no_users=500, backend='loss_sim', init_budget=1000)
]

%store bfs_yloss_cfs

target position loop:   0%|          | 0/4 [00:00<?, ?it/s]

The backend used is: BFS


target position loop: 20it [10:02, 30.12s/it]              

target position loop: 30it [16:37, 32.95s/it]

target position loop: 40it [21:32, 31.92s/it]

...

target position loop: 20it [07:38, 22.94s/it]              

target position loop: 30it [12:35, 24.95s/it]

target position loop: 40it [16:30, 24.51s/it]

target position loop: 40it [19:19, 28.98s/it]


Stored 'bfs_yloss_cfs' (list)


In [ ]:
print('Running BFS strategy with fixed ordering...')

bfs_fixed_yloss_cfs = [
    _find_cfs(test, pretrained_models['lstm'], [3, 5, 7], no_users=500, backend='fixed_loss_sim', init_budget=1000),
    _find_cfs(test, pretrained_models['pooling'], [3, 5, 7], no_users=500, backend='fixed_loss_sim', init_budget=1000),
]

%store bfs_fixed_yloss_cfs

## Utilize similarities based on yloss with DFS backward search

In [232]:
backend='dfs_loss_sim'
dfs_yloss_cfs = [
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [3, 5, 7], no_users=500, init_budget=1000),
#     _find_cfs(test, pretrained_models['pooling'], [3, 5, 7], no_users=500, backend='dfs_loss_sim', init_budget=1000)
]

%store dfs_yloss_cfs

target position loop:   0%|          | 0/3 [00:00<?, ?it/s]

The backend used is: DFS


target position loop: 20it [00:28,  1.44s/it]              

target position loop: 30it [00:45,  1.49s/it]

target position loop: 30it [00:59,  1.97s/it]

Stored 'dfs_yloss_cfs' (list)


In [ ]:
backend='dfs_fixed_loss_sim'
dfs_fixed_yloss_cfs = [
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [3, 5, 7], no_users=500, init_budget=1000),
    _find_cfs(test, pretrained_models['pooling'], get_backend_strategy(backend), [3, 5, 7], no_users=500, init_budget=1000)
]

%store dfs_fixed_yloss_cfs

In [ ]:
backend='bestFS_loss'
bestfs_yloss_cfs = [
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [3, 5, 7], no_users=500, init_budget=1000),
#     _find_cfs(test, pretrained_models['pooling'], [3, 5, 7], no_users=500, backend='bestFS_loss', init_budget=1000)
]

%store bestfs_yloss_cfs

In [ ]:
print('Running BestFS strategy with fixed ordering...')

bestfs_fixed_yloss_cfs = [
    _find_cfs(test, pretrained_models['lstm'], [3, 5, 7], no_users=500, backend='bestFS_fixed_loss', init_budget=1000),
    _find_cfs(test, pretrained_models['pooling'], [3, 5, 7], no_users=500, backend='bestFS_fixed_loss', init_budget=1000)
]

%store bestfs_fixed_yloss_cfs

In [ ]:
backend='topdown_loss'
topdown_bestfs_yloss_cfs = [
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [3, 5, 7], no_users=500, init_budget=1000),
#     _find_cfs(test, pretrained_models['pooling'], [3, 5, 7], no_users=500, backend='topdown_loss', init_budget=1000)
]

%store topdown_bestfs_yloss_cfs

In [75]:
backend='bidirectional'
bidirectional_cfs = [
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [1, 3, 5, 7], no_users=6000, init_budget=1000, alpha=1e-3, normalization='default'),
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [1, 3, 5, 7], no_users=6000, init_budget=1000, alpha=0.5, normalization='default'),
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [1, 3, 5, 7], no_users=6000, init_budget=1000, alpha=0.999, normalization='default'),
]

%store bidirectional_cfs

target position loop:   0%|          | 0/4 [00:00<?, ?it/s]

The backend used is: BiDirectional


users loop:   0%|          | 0/6000 [00:00<?, ?it/s]

target position loop: 20it [1:44:19, 312.97s/it]           

users loop:   0%|          | 0/6000 [00:00<?, ?it/s]

...

users loop:   0%|          | 0/6000 [00:00<?, ?it/s]

target position loop: 40it [3:58:44, 389.55s/it]

users loop:   0%|          | 0/6000 [00:00<?, ?it/s]

target position loop: 40it [5:18:24, 477.60s/it]


Stored 'bidirectional_cfs' (list)


In [72]:
backend='brute_force'
brute_force_cfs = [
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [1, 3, 5, 7], no_users=6000, init_budget=100000),
]

%store brute_force_cfs

target position loop:   0%|          | 0/4 [00:00<?, ?it/s]

The backend used is: BruteForce


target position loop: 20it [2:59:30, 538.52s/it]           

target position loop: 30it [4:11:08, 505.90s/it]

target position loop: 40it [4:28:50, 385.98s/it]

target position loop: 40it [4:33:42, 410.56s/it]


Stored 'brute_force_cfs' (list)


In [77]:
backend='combo'
combo_cfs = [
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [1, 3, 5, 7], no_users=6000, init_budget=1000, alpha=1e-3, normalization='default'),
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [1, 3, 5, 7], no_users=6000, init_budget=1000, alpha=0.5, normalization='default'),
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [1, 3, 5, 7], no_users=6000, init_budget=1000, alpha=0.999, normalization='default'),
]

%store combo_cfs

target position loop:   0%|          | 0/4 [00:00<?, ?it/s]

The backend used is: Combo


users loop:   0%|          | 0/6000 [00:00<?, ?it/s]

target position loop: 20it [2:34:47, 464.38s/it]           

users loop:   0%|          | 0/6000 [00:00<?, ?it/s]

...

users loop:   0%|          | 0/6000 [00:00<?, ?it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

target position loop: 40it [5:18:15, 519.91s/it]

users loop:   0%|          | 0/6000 [00:00<?, ?it/s]

target position loop: 40it [7:05:47, 638.69s/it]


Stored 'combo_cfs' (list)
